In [15]:
from langchain.schema import SystemMessage
import pandas as pd
from datetime import datetime

In [4]:
from langchain.chat_models import ChatOpenAI


#Changes

In [8]:
import re
from datetime import datetime, timedelta

In [ ]:
import sqlite3
import langchain
from datetime import datetime
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

from langchain_google_genai import ChatGoogleGenerativeAI
import streamlit as st
from langchain.agents import Tool
from langchain.agents import initialize_agent, AgentType
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import json
from datetime import datetime, timedelta
import requests

from rich.console import Console
from rich.markdown import Markdown

API_KEY = "Your API Key"

chat_model = ChatGoogleGenerativeAI(
    api_key=API_KEY, model="gemini-1.5-flash", temperature=0.6
)
console = Console()

#Changes

In [9]:

def parse_nlp_input(user_input):
    """
    Parse user input to identify actions like adding expenses or querying past expenses.
    """
    user_input = user_input.lower()
    today = datetime.now()
    parsed_data = {"action": None, "description": None, "amount": None, "start_date": None, "end_date": None}

    # Check if the input is about adding an expense
    expense_match = re.search(r"spent\s+(\d+)\s+on\s+(.*?)\s+(today|yesterday|(\d+)\s+days ago)", user_input)
    if expense_match:
        parsed_data["action"] = "add_expense"
        parsed_data["amount"] = float(expense_match.group(1))
        parsed_data["description"] = expense_match.group(2)
        if expense_match.group(3) == "today":
            parsed_data["start_date"] = parsed_data["end_date"] = today
        elif expense_match.group(3) == "yesterday":
            yesterday = today - timedelta(days=1)
            parsed_data["start_date"] = parsed_data["end_date"] = yesterday
        elif expense_match.group(4):
            days_ago = int(expense_match.group(4))
            parsed_data["start_date"] = parsed_data["end_date"] = today - timedelta(days=days_ago)
        return parsed_data

    # Check if the input is a query
    if "expenses" in user_input:
        parsed_data["action"] = "query_expenses"
        if "today" in user_input:
            parsed_data["start_date"] = parsed_data["end_date"] = today
        elif "yesterday" in user_input:
            yesterday = today - timedelta(days=1)
            parsed_data["start_date"] = parsed_data["end_date"] = yesterday
        elif "last week" in user_input:
            parsed_data["start_date"] = today - timedelta(days=7)
            parsed_data["end_date"] = today
        elif "this month" in user_input:
            parsed_data["start_date"] = today.replace(day=1)
            parsed_data["end_date"] = today
        return parsed_data

    # Default action: ask for clarification
    parsed_data["action"] = "clarify"
    return parsed_data


In [10]:
def get_expenses(start_date=None, end_date=None):
    """
    Retrieve expenses from the database, optionally filtering by date range.
    """
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()

    if start_date and end_date:
        query = """
            SELECT description, amount, timestamp
            FROM expenses
            WHERE timestamp BETWEEN ? AND ?
        """
        cursor.execute(query, (start_date.strftime("%Y-%m-%d 00:00:00"), end_date.strftime("%Y-%m-%d 23:59:59")))
    else:
        query = "SELECT description, amount, timestamp FROM expenses"
        cursor.execute(query)

    expenses = cursor.fetchall()
    conn.close()
    return expenses

In [11]:
user_query = st.text_input("Ask your question or add an expense (e.g., 'I spent 500 on groceries today'):")

if st.button("Submit"):
    parsed_input = parse_nlp_input(user_query)
    
    if parsed_input["action"] == "add_expense":
        add_expense(
            parsed_input["description"],
            parsed_input["amount"]
        )
        update_total_money(get_total_money() - parsed_input["amount"])
        st.write(f"Expense added: {parsed_input['description']} - ₹{parsed_input['amount']}")
        st.write(f"Remaining balance: ₹{get_total_money()}")
    
    elif parsed_input["action"] == "query_expenses":
        expenses = get_expenses(parsed_input["start_date"], parsed_input["end_date"])
        if not expenses:
            st.write("No expenses found for the specified period.")
        else:
            st.write("### Expenses for the specified period:")
            for desc, amt, ts in expenses:
                st.write(f"- {desc}: ₹{amt} (Added on {ts})")
    
    else:
        st.write("I didn't understand your input. Please rephrase.")


2024-11-23 02:40:05.644 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:05.651 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:05.654 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:05.655 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:05.659 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:05.661 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:05.663 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:05.664 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [12]:
if st.button("Get Savings Advice"):
    ai_input = f"My total money is ₹{get_total_money()}. My expenses are:\n" + \
               "\n".join(f"{desc}: ₹{amt}" for desc, amt, ts in get_expenses())
    ai_response = get_chat_response(ai_input + "\nSuggest ways to save better.")
    st.write("### AI Suggestions:")
    st.write(ai_response)


2024-11-23 02:40:39.955 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:39.957 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:39.958 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:39.959 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 02:40:39.960 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [18]:
system_message = SystemMessage(
    content="""
     
 You are a helpful chatbot designed to assist students in analyzing their financial spendings. Follow the given instructions:
 1. If the user provides their spending details, analyze the data and categorize it into the following:
   - **Essential Expenses**: Rent, utilities, groceries, transportation, etc.
   - **Educational Expenses**: Tuition, books, supplies, courses, etc.
   - **Discretionary Expenses**: Dining out, entertainment, hobbies, etc.
   - **Savings**: Amount set aside for future use.
 2. Provide helpful insights such as:
   - Suggestions to reduce unnecessary expenses.
   - Budgeting tips and ideal percentage allocations (e.g., 50% essentials, 30% discretionary, 20% savings).
   - Alerts if spending in one category is unusually high.
 3. If the user asks about financial topics unrelated to their expenses, respond with, "I specialize in analyzing personal financial spen
 4. If the input is unclear, say, "I didn’t understand your input; could you clarify or provide your expense details?"
 5. Always answer in a friendly tone using markdown format with:
   - **Headings** for structure.
   - **Bullet points** for clarity.
   - **Tables**, if needed, to present categorized expenses clearly.
 6. If the user asks for real-time data or external resources, respond, "I don’t have access to real-time financial data; I can only assi
 **context**:{context}
 Provide a detailed, user-friendly response that combines all relevant data from above.
 
You are a helpful chatbot for analyzing students' spending habits. 
Assist users by tracking their available money and expenses over time.
Assist users by understanding their expense patterns, offering summaries, and suggesting ways to save money.

Respond in a user-friendly and conversational tone.
Provide a detailed, user-friendly response that combines all relevant data from above.

1. Provide a summary of their spending, including category-wise percentages and total expenses.
2. Suggest cost-cutting measures based on spending patterns.
3. Recommend smart financial habits, such as setting budgets or saving goals.


"""
)

# Initialize SQLite3 database
DB_FILE = "spending_tracker.db"

def init_db():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    # Create tables if they don't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS user_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            total_money REAL
        )
    """)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS expenses (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            description TEXT,
            amount REAL,
            timestamp TEXT
        )
    """)
    # Ensure one user record exists
    cursor.execute("INSERT OR IGNORE INTO user_data (id, total_money) VALUES (1, 0)")
    conn.commit()
    conn.close()

# Get total money
def get_total_money():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT total_money FROM user_data WHERE id = 1")
    total_money = cursor.fetchone()[0]
    conn.close()
    return total_money

# Update total money
def update_total_money(amount):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("UPDATE user_data SET total_money = ? WHERE id = 1", (amount,))
    conn.commit()
    conn.close()

# Add an expense
def add_expense(description, amount):
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cursor.execute("INSERT INTO expenses (description, amount, timestamp) VALUES (?, ?, ?)", (description, amount, timestamp))
    conn.commit()
    conn.close()

# Get all expenses
def get_expenses():
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    cursor.execute("SELECT description, amount, timestamp FROM expenses")
    expenses = cursor.fetchall()
    conn.close()
    return expenses

# Interactive chat model function
def get_chat_response(user_input):
    messages = [
        system_message,
        HumanMessage(content=user_input)
    ]
    response = chat_model(messages=messages)
    return response.content

# Streamlit UI for Spending Tracker
st.set_page_config(
    page_title="SpendSave AI",
    page_icon="💰",
    layout="wide"
)

def main_ui():
    col1, col2 = st.columns([2, 1])
    
    with col1:
        st.title("💰 SpendSave AI")
        st.subheader("Your Personal Finance Assistant")
    
    with col2:
        st.markdown(
            f"""
            <div style='background-color: #f0f2f6; padding: 20px; border-radius: 10px; text-align: center;'>
                <h3>Available Balance</h3>
                <h2>₹{get_total_money():.2f}</h2>
            </div>
            """,
            unsafe_allow_html=True
        )

# Add this function to create the sidebar with recent transactions
def display_recent_transactions():
    st.sidebar.title("Recent Transactions")
    expenses = get_expenses()
    if not expenses:
        st.sidebar.write("No transactions recorded yet.")
    else:
        # Convert expenses to DataFrame for easier manipulation
        df = pd.DataFrame(expenses, columns=['Description', 'Amount', 'Timestamp'])
        # Sort by timestamp in descending order and take last 10
        df = df.sort_values('Timestamp', ascending=False).head(10)
        
        # Display each transaction without timestamp
        for _, row in df.iterrows():
            st.sidebar.markdown(f"**{row['Description']}**: ₹{row['Amount']:.2f}")
            st.sidebar.divider()
            
# Initialize the database
init_db()

# Input total money if not already set
total_money = get_total_money()

if total_money == 0:
    st.write("You haven't provided your total available money yet.")
    user_money = st.number_input("Enter your total available money:", min_value=0, step=1)
    if st.button("Save Money"):
        update_total_money(user_money)
        st.write(f"Total money saved: ${user_money}")
else:
    st.write(f"Your total available money: ${total_money}")

# Input new expenses
expense_description = st.text_input("Enter an expense description:")
expense_amount = st.number_input("Enter the expense amount:", min_value=0, step=1)

if st.button("Add Expense"):
    if expense_amount > total_money:
        st.write("Insufficient funds! Please check your total money.")
    else:
        # Update total money and add expense
        update_total_money(total_money - expense_amount)
        add_expense(expense_description, expense_amount)
        st.write(f"Expense added: {expense_description} - ${expense_amount}")
        st.write(f"Remaining money: ${get_total_money()}")

# Show expense history
st.write("### Expense History")
expenses = get_expenses()
if not expenses:
    st.write("No expenses recorded yet.")
else:
    for desc, amt, ts in expenses:
        st.write(f"- {desc}: ${amt} (Added on {ts})")

# Generate AI insights
if st.button("Analyze Spending"):
    formatted_expenses = "\n".join(
        f"{desc}: ${amt} (Added on {ts})" for desc, amt, ts in expenses
    )
    ai_input = f"""
    My current available money is ${get_total_money()}.
    My expense history is as follows:
    {formatted_expenses}.
    Please analyze my spending and suggest improvements.
    """
    ai_response = get_chat_response(ai_input)
    st.write("### AI Analysis")
    st.write(ai_response)

2024-11-23 05:53:15.823 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:53:15.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:53:15.828 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:53:15.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:53:15.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:53:15.830 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:53:15.832 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:53:15.834 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [1]:
# Define a new section for the chatbox interface
def create_chatbox():
    """
    Creates a new chatbox interface for real-time conversation with the financial assistant.
    This will allow users to interact in a more conversational format.
    """
    st.markdown("## 💬 Chat with SmartSpend Bot")
    
    # Chat history to preserve context during the session
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []

    # User input field for the chatbox
    user_message = st.text_input(
        "Ask your financial questions or discuss expenses here:",
        placeholder="E.g., 'How can I save more?', 'What do you think of my expenses?'",
    )
    
    # Submit button to send user input
    if st.button("Send", key="chatbox_send"):
        if user_message:
            # Add user message to chat history
            st.session_state.chat_history.append({"sender": "user", "message": user_message})
            
            # Generate a chatbot response based on the user input and context
            ai_response = generate_chatbox_response(user_message)
            
            # Add chatbot's response to chat history
            st.session_state.chat_history.append({"sender": "bot", "message": ai_response})
    
    # Display the chat history in the interface
    for chat in st.session_state.chat_history:
        if chat["sender"] == "user":
            st.markdown(f"**You:** {chat['message']}")
        else:
            st.markdown(f"**SmartSpend Bot:** {chat['message']}")


def generate_chatbox_response(user_input):
    """
    Generates a response for the new chatbox using the financial assistant context.
    """
    # Get the financial context (initial money, total spent, current balance, etc.)
    expenses = get_expenses()
    total_spent = sum(expense[1] for expense in expenses)
    initial_money = get_total_money() + total_spent

    context = f"""
    Initial money: ₹{initial_money}
    Total spent: ₹{total_spent}
    Current balance: ₹{get_total_money()}

    Expense breakdown:
    {chr(10).join(f'- {desc}: ₹{amt}' for desc, amt, _ in expenses)}
    """
    
    # Create a conversational context for the chatbot
    messages = [
        system_message,
        HumanMessage(content=f"{user_input}\n\nContext:\n{context}")
    ]
    
    # Get a response using the AI chat model
    response = chat_model(messages=messages)
    return response.content


# Main UI integration for the chatbox
def main_ui_with_chatbox():
    # Call the primary UI (expense tracker and tabs)
    create_tabs()
    
    # Add the chatbox interface as a separate section
    st.divider()  # Add a visual divider between sections
    create_chatbox()


# Call the new main UI function to run the app
if __name__ == "__main__":
    init_db()
    main_ui_with_chatbox()


NameError: name 'init_db' is not defined

In [19]:
def get_chat_response(user_input):
    # Get all expenses
    expenses = get_expenses()
    total_spent = sum(expense[1] for expense in expenses)  # Sum of all expense amounts
    initial_money = get_total_money() + total_spent  # Calculate initial money
    
    context = f"""
    Initial money: ₹{initial_money}
    Total spent: ₹{total_spent}
    Current balance: ₹{get_total_money()}
    
    Expense breakdown:
    {chr(10).join(f'- {desc}: ₹{amt}' for desc, amt, _ in expenses)}
    """
    
    messages = [
        system_message,
        HumanMessage(content=user_input + "\n\nContext:\n" + context)
    ]
    response = chat_model(messages=messages)
    return response.content